In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.preprocessing import MinMaxScaler
import time
import tensorflow as tf
from tensorflow import keras
import csv
from scipy.stats import spearmanr
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt
from math import floor
from sklearn import metrics
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Flatten
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
from keras.models import load_model
from pandas.core.common import random_state
from sklearn.model_selection import train_test_split
from scipy.stats import linregress

cityZ=['den_helder', 'esbjerg', 'gedser', 'Got', 'helsinki', 'KlagshamnMalmoe', 'lowestoft', 'oslo', 'RatanUmeaa']
pathdata=r'' #where your predictor and predictand time series are
outpath=r'' #where you want to save your statistics

for icity in cityZ:
    Y = pd.read_csv(pathdata+icity+'_target.csv')
    Y = np.ravel(Y)
    X=pd.read_csv(pathdata+icity+'_predictors.csv')
    X=X.values

    X_tr=X[0:floor(len(Y)*0.7),:] 
    Y_tr=Y[0:floor(len(Y)*0.7)]
    X_te=X[floor(len(Y)*0.7)+1:floor(len(Y)*0.85),:] 
    Y_te=Y[floor(len(Y)*0.7)+1:floor(len(Y)*0.85)] 
    X_va=X[floor(len(Y)*0.85)+1:-1,:] 
    Y_va=Y[floor(len(Y)*0.85)+1:-1]

    window = 12 
    X_train = [] #create a matrix of the drivers with current + x previous timesteps
    Y_train = [] #create a list with output of dependent variable MSL
    for i in range(window,len(X_tr)): 
        X_train.append(X_tr[i-window:i,:]) 
        Y_train.append(Y_tr[i-1]) 

    X_train, Y_train = np.array(X_train), np.array(Y_train)

    X_test = [] 
    Y_test = [] 
    for i in range(window,len(X_te)): 
        X_test.append(X_te[i-window:i,:]) 
        Y_test.append(Y_te[i-1]) 

    X_test, Y_test = np.array(X_test), np.array(Y_test)

    X_valid = [] 
    Y_valid = [] 
    for i in range(window,len(X_va)): 
        X_valid.append(X_va[i-window:i,:]) 
        Y_valid.append(Y_va[i-1]) 

    X_valid, Y_valid = np.array(X_valid), np.array(Y_valid)

#train and save 100 models
    RMSE_high=np.zeros(100)
    MAE=np.zeros(100)
    RMSE=np.zeros(100)
    corr=np.zeros(100)
    pvalue=np.zeros(100)

    for i in range(0,100):
        model = Sequential()
        model.add(LSTM(units=33, return_sequences = True)) 
        model.add(Dropout(0.015))
        model.add(LSTM(units=33, return_sequences = True)) 
        model.add(Dropout(0.015)) 
        model.add(LSTM(units=33, return_sequences = True)) 
        model.add(Dropout(0.015))
        model.add(Flatten())
        model.add(Dense(units=1)) 

        optimizer = keras.optimizers.Adam(learning_rate=0.005)
        model.compile(optimizer = optimizer , loss = 'mse')  
        es = EarlyStopping(monitor="val_loss",mode='min',verbose=0, patience=30 , restore_best_weights=True) #callback

        #t0 = time.time()
        history = model.fit(X_train, Y_train, epochs = 120 , batch_size = 100  , validation_data = (X_valid, Y_valid), callbacks = [es], verbose=0)
        model.save(pathdata+icity+'_model_'+str(i)+'.keras')

        Y_pred=model.predict(X_test)
        Y_pred=Y_pred.reshape(-1)
        RMSE_high[i]=np.sqrt(metrics.mean_squared_error(np.where(Y_test>=0.7,Y_test,0),np.where(Y_test>=0.7,Y_pred,0)))
        MAE[i]=metrics.mean_absolute_error(Y_test, Y_pred)
        RMSE[i]=np.sqrt(metrics.mean_squared_error(Y_test, Y_pred))
        _, _, corr[i], pvalue[i], _ = linregress(Y_test, Y_pred)

        del model, optimizer,es,history,Y_pred
        plt.close('all')    

    outfilename=icity+"_noperm_"+str(window)+"_conf3.csv"
    fields = ['RMSE_high','MAE','RMSE','corr','pvalue']   
    with open(str(outpath)+"/"+str(outfilename), 'w') as f:
        f.truncate()
        writer = csv.writer(f, delimiter='\t')
        writer.writerow(fields)
        writer.writerows(zip(RMSE_high,MAE,RMSE,corr,pvalue))  
    del MAE,RMSE,corr,pvalue,RMSE_high
    

In [ ]:
#How to do the prediction once the best model has been selected - an example
i=18 #ID of the best model
pathdata=r'' #where the predictor files are

model = tf.keras.models.load_model(pathdata+city+'_model_'+str(i)+'.keras')
Y_pred=model.predict(X_test)
Y_pred=Y_pred.reshape(-1)
RMSE_high=np.sqrt(metrics.mean_squared_error(np.where(Y_test>=0.7,Y_test,0),np.where(Y_test>=0.7,Y_pred,0)))
RMSE=np.sqrt(metrics.mean_squared_error(Y_test,Y_pred))
_, _, corr, pvalue, _ = linregress(Y_test, Y_pred)
print(str(RMSE_high)+" "+str(RMSE)+" "+str(corr)+" "+str(pvalue))

In [ ]:
#Permutation feature on the best models

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.preprocessing import MinMaxScaler
import time
import tensorflow as tf
from tensorflow import keras
import csv
from scipy.stats import spearmanr
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt
from math import floor
from sklearn import metrics
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Flatten
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
from keras.models import load_model
from pandas.core.common import random_state
from sklearn.model_selection import train_test_split
from scipy.stats import linregress

cityZ=['den_helder', 'esbjerg', 'gedser', 'Got','helsinki', 'KlagshamnMalmoe', 'lowestoft', 'oslo', 'RatanUmeaa'] 
bestmod=[66,51,22,68,14,60,18,39,78] #

pathdata=r''
outpath=r''

comptcity=0
for icity in cityZ:
    Y = pd.read_csv(pathdata+icity+'_target.csv')
    Y = np.ravel(Y)

    RMSE_high=np.zeros(14)
    RMSE=np.zeros(14)
    corr=np.zeros(14)
    pvalue=np.zeros(14)

    for iperm in range(0,14):
        X=pd.read_csv(pathdata+icity+'_predictors.csv')
        X=X.values
        X[:,iperm]=np.random.default_rng().choice(100, size=(X.shape[0], ))/100

        X_te=X[floor(len(Y)*0.15)+1:floor(len(Y)*0.3),:] #validation as the next 15%
        Y_te=Y[floor(len(Y)*0.15)+1:floor(len(Y)*0.3)] 

        
        if icity=='helsinki':
            window=24
        elif icity=='KlagshamnMalmoe':
            window=36
        else:
            window = 12 
        
        X_test = [] 
        Y_test = [] 
        for i in range(window,len(X_te)): 
            X_test.append(X_te[i-window:i,:]) 
            Y_test.append(Y_te[i-1]) 

        X_test, Y_test = np.array(X_test), np.array(Y_test)

        model = tf.keras.models.load_model(icity+'_model_'+str(bestmod[comptcity])+'.keras')
        Y_pred=model.predict(X_test)
        Y_pred=Y_pred.reshape(-1)
        RMSE_high[iperm]=np.sqrt(metrics.mean_squared_error(np.where(Y_test>=0.66,Y_test,0),np.where(Y_test>=0.66,Y_pred,0)))
        RMSE[iperm]=np.sqrt(metrics.mean_squared_error(Y_test,Y_pred))
        _, _, corr[iperm], pvalue[iperm], _ = linregress(Y_test, Y_pred)

    outfilename=icity+"_withperm.csv"
    fields = ['RMSE_high','RMSE','corr','pvalue']   
    with open(str(outpath)+"/"+str(outfilename), 'w') as f:
        f.truncate()
        writer = csv.writer(f, delimiter='\t')
        writer.writerow(fields)
        writer.writerows(zip(RMSE_high,RMSE,corr,pvalue))  
    del RMSE,corr,pvalue,RMSE_high
    comptcity+=1
    